In [1]:
import neptune.new as neptune

import tensorflow as tf
from pprint import pprint
from train_eval import train_support, eval_support
# import dataloaders
# import dl_modules
# import losses

In [2]:
from train import *
from data import *
from model import *

Running on 1 replicas


# Initialize Neptune


In [14]:
run = neptune.init(
    project="10xar-saket/siamese-object-identification",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5ODE2ZDk5ZC00ZTZkLTQ0MmUtYTI2Zi0yOTE3MmNkODNkNmQifQ==",
) 

https://app.neptune.ai/10xar-saket/siamese-object-identification/e/SIAM-1


Info (NVML): Driver Not Loaded. GPU usage metrics may not be reported. For more information, see https://docs.neptune.ai/you-should-know/what-can-you-log-and-display#hardware-consumption


Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


# hyperparameters


In [22]:
lr = 0.000001
batch_size = 1
epochs = 100
save_dir = 'checlpoints/save'
start_epoch = 0
dataset_name = 'Feb2022-demo'
loss_name = 'triplet'

# Load datasets


In [5]:
train_dataset_path = '/media/saket/Elements/datasets/feb2022demo/dataset/train'
dev_dataset_path = '/media/saket/Elements/datasets/feb2022demo/dataset/dev'

train_dataset = DataLoader(2, train_dataset_path ,visualize=True) # dataloaders.multi_frame_loader.VIDEODATA()
dev_dataset = DataLoader(2, dev_dataset_path) # dataloaders.multi_frame_loader.VIDEODATA()

# Create model

In [15]:
with strategy.scope():
    siamese_model = SiameseModel()
restore_model_path = ''
model_name = 'vgg-embedding-siamese'

# Define trainnin and testing steps


In [17]:
def train_step(X, Y, model, compute_cost, optimizer):
    '''
    Train one minibatch
    '''
    with tf.GradientTape() as tape:
        Y_pred = model(X)
        cost = compute_cost(Y, Y_pred)
    
    gradient = tape.gradient(cost, model.trainable_variables)
    optimizer.apply_gradients(zip(gradient, model.trainable_variables))

    return cost, Y_pred

In [18]:
def test_step(X, Y, model, compute_cost):
    '''
    Infer one minibatch
    '''
    Y_pred = model(X)
    cost = compute_cost(Y, Y_pred)

    return cost, Y_pred

# Define / Create loss


In [24]:
def compute_cost(Y, Y_pred, margin=0.5):
    '''
    '''
    # The output of the network is a tuple containing the distances
    # between the anchor and the positive example, and the anchor and
    # the negative example.
    def _contrastive_loss(y_true, y_pred):
        return tfa.losses.contrastive_loss(y_true, y_pred)
        
    loss = tf.convert_to_tensor(0,dtype=tf.float32)
    g = tf.constant(1.0, shape=[1], dtype=tf.float32)
    h = tf.constant(0.0, shape=[1], dtype=tf.float32)
    
    ap_distance, an_distance = Y_pred
    print(ap_distance, an_distance)
    # loss_query_pos = tf.reduce_mean(_contrastive_loss(g, ap_distance))
    # loss_query_neg = tf.reduce_mean(_contrastive_loss(h, an_distance))
    loss = ap_distance - an_distance
    
    # Computing the Triplet Loss by subtracting both distances and
    # making sure we don't get a negative value.
    loss = tf.maximum(loss + margin, 0.0)
    
    return loss

# Define optimizer


In [25]:
optimizer_name = 'Adam'
optimizer_arg = get_optimizer_arg(optimizer_name)
pprint(optimizer_arg)

{'amsgrad': False,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'epsilon': 1e-07,
 'name': 'Adam'}


In [26]:
train_siamese(train_dataset, dev_dataset, lr, batch_size, epochs, siamese_model, 
                             train_step, test_step, compute_cost, 
                             optimizer_name, save_dir = save_dir, 
                             # checkpoint_path = restore_model_path, 
                             start_epoch = start_epoch, # eval_metrics = eval_metrics, 
                             optimizer_arg = optimizer_arg,
                             dataset_name = dataset_name, loss_name = loss_name, 
                             model_name = model_name, run = run, 
                             show_results=-1, save_image=False)


GPUs : []
Total number of training examples = 5
Start epoch - 0 | End epoch - 100
Number of minibatches in training set - 137
Starting training...
tf.Tensor([2.0348078e-05 2.9399609e-05], shape=(2,), dtype=float32) tf.Tensor([2.2770833e-05 9.8893477e-05], shape=(2,), dtype=float32)
tf.Tensor([1.108139e-04 4.730082e-05], shape=(2,), dtype=float32) tf.Tensor([5.4445654e-06 7.1201983e-05], shape=(2,), dtype=float32)


KeyboardInterrupt: 

In [ ]:
siamese_model.trainable_variables